##구글드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##데이터 불러오기

In [ ]:
!cp /content/drive/MyDrive/validation /content/validation.zip

In [ ]:
!unzip /content/FAKE/validation.zip

Archive:  /content/FAKE/FAKE.zip
replace VS_대화체_001/D-A2-C-001-0051.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

##라이브러리 설치

In [ ]:
!pip install spleeter librosa pandas numpy

##데이터 내 음성 추출 및 변수 추출

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from spleeter.separator import Separator

# Spleeter를 사용하여 음성을 분리할 객체 생성 (2 스템: 보컬과 반주 분리)
separator = Separator('spleeter:2stems')

# 데이터 폴더 경로
data_dir = '/content/'
folders = ['REAL', 'FAKE']

# 특성 추출 함수 정의 (1초 간격으로)
def extract_features_per_second(audio_path, sr=22050):
    # Spleeter를 사용하여 보컬과 반주 분리
    output_dir = '/content/spleeter_output'
    separator.separate_to_file(audio_path, output_dir, codec='wav')

    # separated_vocal_path 설정
    separated_vocal_path = os.path.join(output_dir, os.path.splitext(os.path.basename(audio_path))[0], 'vocals.wav')

    # 사람이 발화(보컬)만 로드
    try:
        y, _ = librosa.load(separated_vocal_path, sr=sr)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {separated_vocal_path}")
        return []
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        return []

    duration = librosa.get_duration(y=y, sr=sr)
    features_list = []

    # 전체 오디오를 1초 단위로 자름
    for i in range(0, int(duration)):
        start_sample = i * sr
        end_sample = start_sample + sr
        if end_sample > len(y):
            break
        segment = y[start_sample:end_sample]

        # 각 구간에서 특성 추출
        features = {}
        features['chroma_stft'] = np.mean(librosa.feature.chroma_stft(y=segment, sr=sr))
        features['rms'] = np.mean(librosa.feature.rms(y=segment))
        features['spectral_centroid'] = np.mean(librosa.feature.spectral_centroid(y=segment, sr=sr))
        features['spectral_bandwidth'] = np.mean(librosa.feature.spectral_bandwidth(y=segment, sr=sr))
        features['rolloff'] = np.mean(librosa.feature.spectral_rolloff(y=segment, sr=sr))
        features['zero_crossing_rate'] = np.mean(librosa.feature.zero_crossing_rate(y=segment))

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=20)
        for j in range(1, 21):
            features[f'mfcc{j}'] = np.mean(mfccs[j-1])

        features['time_frame'] = i  # 현재 시간 프레임 (초)
        features_list.append(features)

    return features_list

# 모든 오디오 파일에 대해 특성 추출
data = []

for folder in folders:
    folder_path = os.path.join(data_dir, folder)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.wav') or file_name.endswith('.mp3'):
            file_path = os.path.join(folder_path, file_name)
            features_per_second = extract_features_per_second(file_path)
            for features in features_per_second:
                features['LABEL'] = folder  # 레이블 추가 (REAL or FAKE)
                data.append(features)

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# 데이터프레임 확인
print(df.head())

# CSV 파일로 저장
output_path = '/content/drive/MyDrive/hitpaw_test.csv'
df.to_csv(output_path, index=False)
print(f"추출된 특성들이 포함된 CSV 파일이 저장되었습니다: {output_path}")
